<a href="https://colab.research.google.com/github/BrandonKTC/Statistic_python/blob/main/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Business Analyst course/Segmentation/RFM

/content/drive/MyDrive/Business Analyst course/Segmentation/RFM


In [2]:
# import libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [6]:
# data
data = pd.read_csv("online_shoppers.csv",
                   encoding="ISO-8859_1").dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Data Preparation

In [8]:
# Creating Sales column
data['sales'] = data["Quantity"] * data["UnitPrice"]
data.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.3


In [12]:
# Dates - Transform Date variable
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

# Get last date available
snapshot_date = data["InvoiceDate"].max() + timedelta(days = 1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [ ]:
# Aggregate on Customer Level
df = data.groupby("CustomerID").aggregate({
    'InvoiceDate': lambda x : (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'sales': 'sum'
})

In [16]:
# Create basket / Monetary variable
df["monetary"] = df.sales / df.InvoiceNo
df.head()

,InvoiceDate,InvoiceNo,sales,monetary
CustomerID,,,,
12346.0,326,2,0.00,0.000000
12347.0,2,182,4310.00,23.681319
12348.0,75,31,1797.24,57.975484
12349.0,19,73,1757.55,24.076027
12350.0,310,17,334.40,19.670588


In [17]:
# Dropping Sales Variable
df.drop("sales", axis=1, inplace=True)
df.head()

,InvoiceDate,InvoiceNo,monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484
12349.0,19,73,24.076027
12350.0,310,17,19.670588


In [18]:
# Changing Variables names
df.rename(columns = {"InvoiceDate":'Recency',
                     'InvoiceNo': 'Frequency'}, inplace=True)
df.head(1)

,Recency,Frequency,monetary
CustomerID,,,
12346.0,326,2,0.0


# RFM Preparation

In [23]:
# Create Frequency Groups
df["F"] = pd.qcut(df.Frequency,
                  4,
                  range(1, 5, 1))
df["M"] = pd.qcut(df.monetary,
                  4,
                  range(1,5,1))
df["R"] = pd.qcut(df.Recency,
                  4,
                  range(4, 0, -1))
df.head()

,Recency,Frequency,monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2
12349.0,19,73,24.076027,3,4,3
12350.0,310,17,19.670588,1,3,1


In [25]:
# RFM Score
df["RFM"] = df[["R", "F", "M"]].sum(axis=1)
df.head(2)

,Recency,Frequency,monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12


In [28]:
# Create the RFM Function
def rfm_segment(df):
  if df["RFM"] >= 11:
    return "SuperStar"
  elif df["RFM"] >= 8 and df["RFM"] < 11:
    return "Future Champion"
  elif df["RFM"] >= 6 and df["RFM"] <  8:
    return 'High Potential'
  else:
    return 'Low Relevance'
  

In [29]:
df["RFM_level"] = df.apply(rfm_segment, axis=1)
df.head()

,Recency,Frequency,monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
12346.0,326,2,0.000000,1,1,1,3,Low Relevance
12347.0,2,182,23.681319,4,4,4,12,SuperStar
12348.0,75,31,57.975484,2,4,2,8,Future Champion
12349.0,19,73,24.076027,3,4,3,10,Future Champion
12350.0,310,17,19.670588,1,3,1,5,Low Relevance


In [30]:
# Looking into the segments
df.groupby("RFM_level").agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'monetary': ['mean', 'count']
})

Recency   Frequency   monetary      
                       mean        mean       mean count
RFM_level                                               
Future Champion   32.007919  135.835973  30.169670  1768
High Potential   118.708869   39.213947  38.033024  1477
Low Relevance    221.483400   22.811421  -0.586430   753
SuperStar          9.973262  244.852941  45.483949   374